In [ ]:
import os
os.chdir('..')
os.chdir('..')
import scvelo as scv
import scanpy as sc
import cell2fate as c2f
import pickle as pickle
from eval_utils import cross_boundary_correctness
from datetime import datetime
import pandas as pd
import numpy as np
from os.path import exists
import matplotlib.pyplot as plt
import torch
import unitvelo as utv
method = 'Cell2fateDynamicalModelTmax150Gamma05Lambda1_Init-tdelta0_400epochs_10samples'
data_dir = '/nfs/team283/aa16/data/fate_benchmarking/benchmarking_datasets/'
save_dir = '/nfs/team283/aa16/data/fate_benchmarking/benchmarking_results/'
datasets = ['Pancreas_with_cc',  'DentateGyrus' , 'MouseBoneMarrow', 'MouseErythroid', 'HumanBoneMarrow']
n_genes_list = np.array((2000, 3000))
n_counts_list = np.array((10, 20))

for i in range(len(n_genes_list)):
    for j in range(len(n_counts_list)):
        for k in (3,0,1,2,4):
            print(i)
            print(j)
            print(k)
            dataset = datasets[k]
            n_genes = n_genes_list[i]
            min_counts = n_counts_list[j]
            model_index = str(i) + '-' + str(j) + '-' + str(k)
            save_name = method + '_'
            if exists(save_dir + save_name + '_CBDC_fullBenchmark.csv'):
                tab = pd.read_csv(save_dir + save_name + '_CBDC_fullBenchmark.csv', index_col = 0)
                if model_index in tab.index:
                    continue
            adata = sc.read_h5ad(data_dir + dataset + '/' + dataset + '_anndata.h5ad')
            adata = c2f.utils.get_training_data(adata, cells_per_cluster = 10**6, cluster_column = 'clusters',
                                            remove_clusters = [], min_shared_counts = min_counts, n_var_genes= n_genes)
            c2f.Cell2fate_DynamicalModel.setup_anndata(adata, spliced_label='spliced', unspliced_label='unspliced')    
            n_modules = c2f.utils.get_max_modules(adata)
            t_delta = torch.ones(n_modules) * 0.001
            t_delta[0] = 150 - 0.1
            mod = c2f.Cell2fate_DynamicalModel(adata,
                                               n_modules = n_modules,
                                               Tmax_prior={"mean": 150., "sd": 50.},
                                               activation_rate_hyp_prior={"mean_hyp_prior_mean": 1., "mean_hyp_prior_sd": 0.33,
                                               "sd_hyp_prior_mean": 0.33, "sd_hyp_prior_sd": 0.1},
                                               degredation_rate_hyp_prior={"mean": 0.5, "alpha": 5.0,
                                                                        "mean_hyp_alpha": 10.0, "alpha_hyp_alpha": 20.0},
                                              init_vals = {'t_delta': t_delta})   
            mod.train(max_epochs = 400)
            sample_kwarg = {"num_samples": 10, "batch_size" : adata.n_obs,
                 "use_gpu" : True, 'return_samples': False}
            adata = mod.export_posterior(adata, sample_kwargs = sample_kwarg)
            mod.compute_and_plot_total_velocity_scvelo(adata, save = False, delete = False)
            # Calculate performance metrics:
            file = open(data_dir + dataset + '/' + dataset + '_groundTruth.pickle' ,'rb')
            ground_truth = pickle.load(file)
            metrics = utv.evaluate(adata, ground_truth, 'clusters', 'velocity')
            cb_score = [np.mean(metrics['Cross-Boundary Direction Correctness (A->B)'][x])
                        for x in metrics['Cross-Boundary Direction Correctness (A->B)'].keys()]
            if exists(save_dir + save_name + '_CBDC_fullBenchmark.csv'):
                tab = pd.read_csv(save_dir + save_name + '_CBDC_fullBenchmark.csv', index_col = 0)
            else:
                c_names = ['CBDC']
                tab = pd.DataFrame(columns = c_names)
            tab.loc[model_index, 'CBDC'] = np.mean(cb_score)
            tab.to_csv(save_dir + save_name + '_CBDC_fullBenchmark.csv')
tab = pd.read_csv(save_dir + save_name + '_CBDC_fullBenchmark.csv', index_col = 0)
tab.loc['AVERAGE', 'CBDC'] = np.mean(tab['CBDC'])
tab.to_csv(save_dir + save_name + '_CBDC_fullBenchmark.csv')

Global seed set to 0


(Running UniTVelo 0.2.5)
2023-10-29 17:57:56
0
0
3
Keeping at most 1000000 cells per cluster
Filtered out 46616 genes that are detected 10 counts (shared).
Extracted 2000 highly variable genes.
Leiden clustering ...
Number of Leiden Clusters: 9
Maximal Number of Modules: 10


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Sampling global variables, sample: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:05<00:00,  1.75it/s]
Computing total RNAvelocity ...


  0%|          | 0/9815 [00:00<?, ?cells/s]

...

Number of Leiden Clusters: 13
Maximal Number of Modules: 14
Sampling global variables, sample: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:02<00:00,  3.60it/s]
Computing total RNAvelocity ...


  0%|          | 0/3696 [00:00<?, ?cells/s]

# Cross-Boundary Direction Correctness (A->B)
{('Ngn3 high EP', 'Pre-endocrine'): -0.7180715831294715, ('Pre-endocrine', 'Alpha'): 0.2036540592398334, ('Pre-endocrine', 'Beta'): 0.3860576184379115, ('Pre-endocrine', 'Delta'): 0.14903527737695813, ('Pre-endocrine', 'Epsilon'): -0.4307328965246739}
Total Mean: -0.08201150491988848
# In-cluster Coherence
{'Alpha': 0.89533234, 'Beta': 0.90740997, 'Delta': 0.89884406, 'Ductal': 0.9948488, 'Epsilon': 0.920336, 'Ngn3 high EP': 0.99034524, 'Ngn3 low EP': 0.99548465, 'Pre-endocrine': 0.9084809}
Total Mean: 0.9388852119445801
0
1
1
Keeping at most 1000000 cells per cluster
Filtered out 10340 genes that are detected 20 counts (shared).
Extracted 2000 highly variable genes.
Leiden clustering ...
         Falling back to preprocessing with `sc.pp.pca` and default params.


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Number of Leiden Clusters: 14
Maximal Number of Modules: 16
Epoch 259/400:  64%|███████████████████████████████████████████████████████████████████████████████████████████████████▎                                                      | 258/400 [03:53<02:07,  1.11it/s, v_num=1, elbo_train=4.78e+6]